# Verifying false-positive rates by simulation

This notebook contains simulations to show that the permutation-based approach to group-sequential testing used in this package effectively controls the false-positive rate, even when analyzing the data sequentially (i.e. stopping data collection when the pattern of interest is significant, and continuing otherwise). 

In these simulations, we'll use functions from the `niseq.max_test` module, which perform permutation _t_-tests with a $t$-max correction for multiple comparisons, to illustrate how `niseq` can control the familywise error rate across multiple looks during data collection. While we don't show simulations for our cluster-based tests here (those take a lot longer to run), note that the same backend functions, `niseq._permutation.generate_permutation_dist` and `niseq._permutation.find_thresholds`, are used under the hood of every statistical test implemented in `niseq`, so the permutation scheme scrutinized here is the exact same as that used in our cluster-level tests.

Note that these simulations will only give us approximate results. Feel free to increase `N_SIMULATIONS` to get something closer to the true false-positive rate.

In [1]:
from mne.parallel import parallel_func
from niseq.max_test import (
    sequential_permutation_t_test_1samp, 
    sequential_permutation_test_indep
)
import numpy as np

N_SIMULATIONS = 1000

On each simulation below, we'll generate null data and pretend we look at it five times throughout the intended course of data collection, and we'll compare the false positive rates attained when we reject the null hypothesis whenever $p \leq 0.05$ and $p \leq \alpha_\text{adjusted}$ at at least one look time.

In [2]:
def one_simulation(seed, 
                tail = 0, 
                indep = False,
                look_times = np.linspace(100, 500, 5).astype(int),
                n_tests = 100):
    
    # generate null data 
    rng = np.random.default_rng(seed)
    x = rng.normal(loc = 0, size = (look_times[-1], n_tests))
    
    # run sequential test
    if indep:
        conds = rng.choice([0, 1], look_times[-1])
        _, p, adj_alpha, _ = sequential_permutation_test_indep(
            x, conds, look_times, n_max = look_times[-1], 
            tail = tail,
            verbose = 0,
            seed = seed
        ) 
    else:
        _, p, adj_alpha, _ = sequential_permutation_t_test_1samp(
            x, look_times, n_max = look_times[-1], 
            tail = tail,
            verbose = 0,
            seed = seed
        ) 
        
    # reject if p-val crosses sig threshold at any look time
    return np.array([np.any(p < .05), np.any(p < adj_alpha)]) 


def run_simulations(n_simulations, tail = 0, indep = False, n_jobs = -1):
    parallel, p_func, _ = parallel_func(one_simulation, n_jobs)
    out = parallel(p_func(seed, tail, indep) for seed in range(n_simulations))
    rejections = np.stack(out)
    fpr = rejections.mean(0)
    print('False positive rate without correction: ' + str(fpr[0]))
    print('False positive rate *with* correction: ' + str(fpr[1]))

## One Sample Test

In [3]:
run_simulations(N_SIMULATIONS, indep = False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 874 tasks      | elapsed:  5.8min


False positive rate without correction: 0.163
False positive rate *with* correction: 0.042


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  6.5min finished


If we don't correct for sequential looks using our alpha spending approach, rejecting the null and stopping data collection as soon as we see $p \leq 0.05$ but continuing to collect data otherwise, we end up with an inflated false positive rate. But using our adjusted thresholds, familywise error rates are contained below our target $\alpha = 0.05$, even if we stop data collection on the first look where the test statistic passes the significance threshold! Thus, as long as we can specify a reasonable $n_\text{max}$ we're willing to collect, we can use this procedure to determine our sample size adaptively.

## Independent Sample Test

In [4]:
# simulate false-positive rate for two-sample test
run_simulations(N_SIMULATIONS, indep = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 874 tasks      | elapsed:  8.2min


False positive rate without correction: 0.175
False positive rate *with* correction: 0.049


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  9.4min finished


## Note

[Similar simulations](https://github.com/john-veillette/niseq/blob/main/niseq/tests/test_permutation.py) are run whenever new code is added to `niseq` as part our continuous integration pipeline to ensure the permutation scheme continues to control the false positive rate following any changes. 